In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
# import joblib
# import time
# from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore') #warning 문구 제거
# pd.set_option('display.max_columns', None)

In [3]:
#데이터 불러오기
visit = pd.read_csv('data/visit_area_processing.csv')
travel = pd.read_csv('data/travel_result.csv')
traveler_master = pd.read_csv('data/traveler_master_result.csv')
move = pd.read_csv('data/move_result.csv')
code = pd.read_csv('data/codeB.csv')
sgg = pd.read_csv('data/sigungu_code.csv')


In [4]:
# 집, 친지집, 사무실 제거
visit_use = visit[~visit['VISIT_AREA_TYPE_CD'].isin([21,22,23])].reset_index

In [5]:
# 시군구 코드
# 'SGG_CD2'의 NaN 값을 0으로 대체한 후 int로 변환
sgg['SGG_CD2'] = sgg['SGG_CD2'].fillna(0).astype(int)

# 'SGG_CD1'와 'SGG_CD2'를 합쳐서 'TRAVEL_LIKE_SGG' 컬럼을 매핑
sgg['SGG_CODE'] = sgg['SGG_CD1'].astype(str) + sgg['SGG_CD2'].astype(str)
sgg['SGG_NAME'] = sgg['SIDO_NM'] + ' ' + sgg['SGG_NM']

# 'SGG_CD1'과 'SIDO_NM' 매핑 딕셔너리 생성
sido_mapping = dict(zip(sgg['SGG_CD1'], sgg['SIDO_NM']))

# 'SGG_CODE'와 'SGG_NAME' 매핑 딕셔너리 생성
sgg_mapping = dict(zip(sgg['SGG_CODE'], sgg['SGG_NAME']))

# SGG_CODE 목록
sgg_code_list = sgg['SGG_CODE'].astype(str).tolist()

def find_closest_sgg_code(sgg_code, sgg_code_list):
    try:
        sgg_code = int(sgg_code)
        sgg_code_list = np.array([int(code) for code in sgg_code_list])
        closest_index = (np.abs(sgg_code_list - sgg_code)).argmin()
        return str(sgg_code_list[closest_index]).zfill(5)  # 원래 형식을 맞추기 위해 5자리로 채움
    except ValueError:
        return None

# TRAVEL_LIKE_SIDO 컬럼에 매핑 적용
for i in range(1, 4):
    col_name = f'TRAVEL_LIKE_SIDO_{i}'
    traveler_master[col_name] = traveler_master[col_name].map(sido_mapping)

# TRAVEL_LIKE_SGG 컬럼에 매핑 적용 및 근처 값 대체
for i in range(1, 4):
    col_name = f'TRAVEL_LIKE_SGG_{i}'
    for index, value in traveler_master[col_name].astype(str).items():
        if value in sgg_mapping:
            traveler_master.at[index, col_name] = sgg_mapping[value]
        else:
            closest_code = find_closest_sgg_code(value, sgg_code_list)
            if closest_code:
                traveler_master.at[index, col_name] = sgg_mapping.get(closest_code, 'Unknown')
            else:
                traveler_master.at[index, col_name] = 'Unknown'

# 주소가 3단어인 경우를 처리
def convert_to_two_words(address):
    words = address.split()
    if len(words) == 3:
        return ' '.join(words[:2])
    return address

# 주소 변환 적용
traveler_master['TRAVEL_LIKE_SGG_1'] = traveler_master['TRAVEL_LIKE_SGG_1'].apply(convert_to_two_words)

# (매핑용 코드)
code_tsy = code[code['cd_a'] == 'TSY'][['cd_b','cd_nm']]
code_tsy['cd_b'] = code_tsy['cd_b'].astype(int)
code_tsy = dict(zip(code_tsy['cd_b'], code_tsy['cd_nm']))

# # 여행스타일 매핑
traveler_master['STYL1'] = traveler_master['TRAVEL_STYL_1'].map(code_tsy)

# 여행master, 여행 merge
merge_trav = pd.merge(traveler_master, travel, on = 'TRAVELER_ID')

In [6]:
# 방문지 정보 추가 매핑
use_data = pd.merge(merge_trav, visit, on = 'TRAVEL_ID')

In [7]:
## 여행지 추가를 위한 추가 데이터

# 방문지 정보 여행지 정리
id_check = travel[['TRAVEL_ID','TRAVELER_ID']]
id_check = id_check.drop_duplicates().reset_index(drop=True)

# 방문지 정보 데이터 처리
visit_use = visit.dropna(subset=['ROAD_NM_ADDR'])
visit_use = visit_use.reset_index(drop = True)

# 시도/군구 변수 생성
visit_use['sido'] = visit_use['ROAD_NM_ADDR'].apply(lambda x: x.split(' ')[0] if isinstance(x, str) and len(x.split(' ')) > 1 else np.nan)
visit_use['gungu'] = visit_use['ROAD_NM_ADDR'].apply(lambda x: x.split(' ')[1] if isinstance(x, str) and len(x.split(' ')) > 1 else np.nan)

list(visit_use['sido'].unique())
sgg['SIDO_NM'].unique()
sgg_list = list(sgg['SGG_NM'].dropna().unique())
sido_all = list(sgg['SIDO_NM'].unique())

# 각 시군구 데이터에서 앞 단어만 추출
sgg_all = [location.split()[0] for location in sgg_list]

visit_use = visit_use[visit_use['sido'].isin(sido_all)]
visit_use = visit_use[visit_use['gungu'].isin(sgg_all)]
visit_use = visit_use.dropna(subset=['sido']).reset_index(drop=True)
visit_use = visit_use.dropna(subset=['gungu']).reset_index(drop=True)

# mapping = {
#     '서울': '서울특별시',
#     '부산': '부산광역시',
#     '대구': '대구광역시',
#     '인천': '인천광역시',
#     '광주': '광주광역시',
#     '대전': '대전광역시',
#     '울산': '울산광역시',
#     '세종': '세종특별자치시',
#     '경기': '경기도',
#     '강원': '강원도',
#     '충북': '충청북도',
#     '충남': '충청남도',
#     '전북': '전라북도',
#     '전남': '전라남도',
#     '경북': '경상북도',
#     '경남': '경상남도',
#     '제주': '제주특별자치도',
# }

# visit_use['sido'] = visit_use['sido'].map(mapping)

visit_use['LOCATION'] = visit_use['sido'] + ' ' + visit_use['gungu']

In [8]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 범주형 변수를 숫자로 인코딩
label_encoders = {
    'GENDER': LabelEncoder(),
    'TRAVEL_STATUS_RESIDENCE': LabelEncoder(),
    'STYL1': LabelEncoder(),
    'TRAVEL_LIKE_SGG_1': LabelEncoder()
}

for col, encoder in label_encoders.items():
    use_data[col] = encoder.fit_transform(use_data[col])

# 입력 데이터와 출력 데이터
X = use_data[['AGE_GRP', 'GENDER', 'TRAVEL_STATUS_RESIDENCE', 'STYL1']]
y = use_data['TRAVEL_LIKE_SGG_1']

# # 빈도수 계산 및 가중치 설정
# class_counts = y.value_counts()
# class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
# use_data['weight'] = use_data['TRAVEL_LIKE_SGG_1'].map(class_weights)


# 데이터 분할
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.2, random_state=42)


from imblearn.over_sampling import RandomOverSampler

rds = RandomOverSampler(random_state=42)
X_train_resample, y_train_resample = rds.fit_resample(X_train,y_train)
print(f'shape : {X_train.shape}, {X_train_resample.shape}')

shape : (62764, 4), (2693481, 4)


In [15]:

# Destination 예측 모델
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=7, verbose=10)
model.fit(X_train, y_train)

# Destination 예측
y_pred = model.predict(X_test)

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 결과 출력
print("Destination 모델 성능:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print()


0:	learn: 4.1156088	total: 929ms	remaining: 1m 32s
10:	learn: 3.2229356	total: 15.3s	remaining: 2m 3s
20:	learn: 2.9882836	total: 26.2s	remaining: 1m 38s
30:	learn: 2.8350283	total: 35.2s	remaining: 1m 18s
40:	learn: 2.7395062	total: 44.9s	remaining: 1m 4s
50:	learn: 2.6710942	total: 54.8s	remaining: 52.6s
60:	learn: 2.6147773	total: 1m 6s	remaining: 42.7s
70:	learn: 2.5677916	total: 1m 19s	remaining: 32.4s
80:	learn: 2.5391470	total: 1m 29s	remaining: 21s
90:	learn: 2.5047798	total: 1m 40s	remaining: 9.97s
99:	learn: 2.4718507	total: 1m 51s	remaining: 0us
Destination 모델 성능:
Accuracy: 0.2647
Precision: 0.2869
Recall: 0.2647
F1 Score: 0.2164



In [9]:
# 빈도수 계산 및 가중치 설정
class_counts = y.value_counts()
class_weights = {cls: 1.0 / count for cls, count in class_counts.items()}
use_data['weight'] = use_data['TRAVEL_LIKE_SGG_1'].map(class_weights)

# 데이터 분할
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, use_data['weight'], test_size=0.2, random_state=42)

# from imblearn.over_sampling import RandomOverSampler

# rds = RandomOverSampler(random_state=42)
# X_train_resample, y_train_resample = rds.fit_resample(X_train,y_train)
# print(f'shape : {X_train.shape}, {X_train_resample.shape}')


In [11]:

# Destination 예측 모델
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=10)
model.fit(X_train, y_train, sample_weight=weights_train)

# Destination 예측
y_pred = model.predict(X_test)

# 평가 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# 결과 출력
print("Destination 모델 성능:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print()

0:	learn: 4.9178882	total: 956ms	remaining: 1m 34s
10:	learn: 3.4598743	total: 12s	remaining: 1m 37s
20:	learn: 2.9529804	total: 23s	remaining: 1m 26s
30:	learn: 2.6682444	total: 35s	remaining: 1m 18s
40:	learn: 2.4666164	total: 44.5s	remaining: 1m 4s
50:	learn: 2.3261808	total: 54.6s	remaining: 52.5s
60:	learn: 2.2069602	total: 1m 5s	remaining: 41.9s
70:	learn: 2.1249134	total: 1m 15s	remaining: 31s
80:	learn: 2.0575712	total: 1m 25s	remaining: 20s
90:	learn: 2.0065537	total: 1m 37s	remaining: 9.64s
99:	learn: 1.9692413	total: 1m 45s	remaining: 0us
Destination 모델 성능:
Accuracy: 0.0658
Precision: 0.0642
Recall: 0.0658
F1 Score: 0.0361



In [18]:
# 새로운 데이터 생성 함수
def generate_random_data(n_samples):
    np.random.seed(43)  # 랜덤 시드 설정

    genders = np.random.choice(['남', '여'], size=n_samples) 
    ages = np.random.choice([20, 30, 40, 50, 60], size=n_samples)  
    residences = np.random.choice(sido_all, size=n_samples)
    travel_styles = {
        'STYL1': np.random.choice(['자연 선호 약간선호', '자연 선호 중간선호', '자연 선호 매우선호', '중립', '도시 선호 약간선호', '도시 선호 중간선호', '도시 선호 매우선호'], size=n_samples)
    }

    data = {
        'GENDER': genders,
        'AGE_GRP': ages,
        'TRAVEL_STATUS_RESIDENCE': residences,
        'STYL1': travel_styles['STYL1']
    }


    new_df = pd.DataFrame(data)
    return new_df

# 새로운 데이터 생성 (예시: 5개의 샘플 생성)
n_samples = 10
random_data = generate_random_data(n_samples)

for col, encoder in label_encoders.items():
    if col in random_data.columns:
        random_data[col] = encoder.transform(random_data[col])

# 새로운 데이터 예측
predicted_destinations = model.predict(random_data)
predicted_destinations_label = label_encoders['TRAVEL_LIKE_SGG_1'].inverse_transform(predicted_destinations)

# 명소 추천 함수 정의
def recommend_attractions(destination, visit_data, top_n=10):
    attractions = visit_data[visit_data['LOCATION'] == destination]
    attraction_counts = attractions['VISIT_AREA_NM'].value_counts().head(top_n)
    top_attractions = attractions[attractions['VISIT_AREA_NM'].isin(attraction_counts.index)]
    return top_attractions.drop_duplicates(subset=['VISIT_AREA_NM'])


# 추천된 여행지와 명소 출력 및 결과 저장
results = []
for i, destination in enumerate(predicted_destinations_label):
    attractions = recommend_attractions(destination, visit_use)
    attractions_names = ', '.join(attractions['VISIT_AREA_NM'].tolist())
    df_gender = label_encoders['GENDER'].inverse_transform([random_data.iloc[i]['GENDER']])[0]
    df_age = random_data.iloc[i]['AGE_GRP']
    df_residence = label_encoders['TRAVEL_STATUS_RESIDENCE'].inverse_transform([random_data.iloc[i]['TRAVEL_STATUS_RESIDENCE']])[0]
    df_STYL = label_encoders['STYL1'].inverse_transform([random_data.iloc[i]['STYL1']])[0]
    print(f"GENDER: {df_gender}, AGE: {df_age}, RESIDENCE: {df_residence}, TRAVEL STYLE: {df_STYL}")
    print(f"추천 여행지: {destination}")
    print(f"추천 명소: {attractions_names}")
    print('==='*10)
    for index,row in attractions.iterrows():
        results.append({
            'GENDER': df_gender,            
            'AGE_GRP': df_age,
            'TRAVEL_STATUS_RESIDENCE': df_residence,
            'STYL1': df_STYL,
            'Predicted_Destination': destination,
            'Recommended_Places': row['VISIT_AREA_NM'],
            'Recommended_Address' : row['ROAD_NM_ADDR'],
            'X_COORD' : row['X_COORD'],
            'Y_COORD' : row['Y_COORD']
    })

# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

GENDER: 남, AGE: 50, RESIDENCE: 충청남도, TRAVEL STYLE: 도시 선호 약간선호
추천 여행지: 대전광역시 유성구
추천 명소: 국립 중앙과학관, 엑스포과학공원 한빛탑, 롯데시티 호텔 대전, 대전 신세계 아트앤사이언스, 오씨칼국수 도룡점, 성심당 DCC점, 대전 어린이 회관, 원조태평소국밥, 유성온천공원 족욕체험장, 유성호텔
GENDER: 남, AGE: 40, RESIDENCE: 대구광역시, TRAVEL STYLE: 자연 선호 약간선호
추천 여행지: 경상북도 영덕군
추천 명소: 고래불해수욕장, 스포트리조트, 영덕해맞이공원, 삼사해상공원, 벌열 메타세쿼이아 숲, 장사상륙작전 전승기념관, 대게 궁, 장사해수욕장, 영덕 해 파랑공원, 그리가다
GENDER: 여, AGE: 20, RESIDENCE: 인천광역시, TRAVEL STYLE: 자연 선호 중간선호
추천 여행지: 인천광역시 남동구
추천 명소: 통 큰 해물 손칼국수 소래포구 본점, 소래포구, 소래포구 종합어시장, 파크 마린 호텔, 인천대공원, 라마다 인천 호텔, 소래습지생태공원, 메가박스 인천 논현, 그랜드 팰리스 호텔 인천, 남동공단 떡볶이
GENDER: 여, AGE: 50, RESIDENCE: 광주광역시, TRAVEL STYLE: 자연 선호 매우선호
추천 여행지: 전라남도 영광군
추천 명소: 불갑사, 백수해안도로, 영광 칠산 다워, 야월 교회, 카페 보리, 국제식당, 백제불교 최초 도래지 부용리, 영광 물 무산 행복 숲 질퍽질퍽 맨발 황톳길, 민속적, 법성 토우
GENDER: 여, AGE: 40, RESIDENCE: 광주광역시, TRAVEL STYLE: 자연 선호 약간선호
추천 여행지: 전라남도 완도군
추천 명소: 방파제, 범바위, 완도타워, 신지명사십리해수욕장, 서편제촬영지, 완도네시아, 지리해수욕장, 카페마르, 신흥해수욕장, 청산도게스트하우스
GENDER: 남, AGE: 50, RESIDENCE: 제주특별자치도, TRAVEL STYLE: 도시 선호 약간선호
추천 여행지: 대전광역

In [19]:
results_df.head()

,GENDER,AGE_GRP,TRAVEL_STATUS_RESIDENCE,STYL1,Predicted_Destination,Recommended_Places,Recommended_Address,X_COORD,Y_COORD
0,남,50,충청남도,도시 선호 약간선호,대전광역시 유성구,국립 중앙과학관,대전광역시 유성구 대덕대로 481,127.375212,36.376069
1,남,50,충청남도,도시 선호 약간선호,대전광역시 유성구,엑스포과학공원 한빛탑,대전광역시 유성구 대덕대로 480,127.388084,36.376629
2,남,50,충청남도,도시 선호 약간선호,대전광역시 유성구,롯데시티 호텔 대전,대전광역시 유성구 엑스포로123번길 33,127.393034,36.376192
3,남,50,충청남도,도시 선호 약간선호,대전광역시 유성구,대전 신세계 아트앤사이언스,대전광역시 유성구 엑스포로 1,127.381377,36.375242
4,남,50,충청남도,도시 선호 약간선호,대전광역시 유성구,오씨칼국수 도룡점,대전광역시 유성구 엑스포로151번길 19,127.394782,36.376333
